# apply 메서드 활용

apply method 는 사용자가 작성한 함수를 한번에 DataFrame 의 각 행과 열에 적용하여 실행할 수 있게 해주는 메서드
함수를 브로드캐스팅해야 하는 경우에 apply method 를 사용한다.

## 간단한 함수 만들기

In [2]:
# 제곱 함수와 n제곱 함수
def my_sq(x):
    return x ** 2

def my_exp(x, n):
    return x ** n

In [3]:
my_sq(4)

16

In [4]:
my_exp(2, 4)

16

## apply method 사용하기 - 기초

### series 와 apply

In [5]:

import pandas as pd

df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
df

,a,b
0,10,20
1,20,30
2,30,40


In [6]:
# my_exp 사용하지 않고 a 열을 제곱하여 얻은 결과
df['a'] ** 2

0    100
1    400
2    900
Name: a, dtype: int64

In [7]:
# apply 메서드와 my_exp 적용
sq = df['a'].apply(my_sq)
sq

0    100
1    400
2    900
Name: a, dtype: int64

In [11]:
# 2개의 인자를 전달받는 n제곱함수 (my_exp) 와 함께 사용
# df['a'] 값이 my_exp 의 첫번째 인자로 넘어간다.
ex = df['a'].apply(my_exp, n=2)
ex

0    100
1    400
2    900
Name: a, dtype: int64

In [13]:
ex = df['a'].apply(my_exp, n=3)
ex

0     1000
1     8000
2    27000
Name: a, dtype: int64

### DataFrame 과 apply

In [14]:
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
df

,a,b
0,10,20
1,20,30
2,30,40


In [16]:
# 1개의 값을 전달받아 출력하는 함수 작성
def print_me(x):
    print(x)

In [17]:
# axis 값을 0 이나 1로 지정하여 함수를 열 또는 행 방향으로 적용
df.apply(print_me, axis=0)

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64


a    None
b    None
dtype: object

In [19]:
df.apply(print_me, axis=1)

a    10
b    20
Name: 0, dtype: int64
a    20
b    30
Name: 1, dtype: int64
a    30
b    40
Name: 2, dtype: int64


0    None
1    None
2    None
dtype: object

In [21]:
# 3개의 인자를 입력받아 평균을 계산하는 함수
def avg_3(x, y, z):
    return (x + y + z) / 3

In [22]:
df.apply(avg_3) # df 가 열단위 데이터로 전달 되었으나 avg_3 함수에서 1개의 인자로 인식한다.

TypeError: ("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')

In [23]:
# avg_3 가 열단위 데이터를 수정할 수 있도록 수정

def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return (x + y + z) / 3

df.apply(avg_3_apply)

a    20.0
b    30.0
dtype: float64

In [24]:
ret = df.apply(avg_3_apply)
type(ret)

pandas.core.series.Series

In [32]:
# 일반적으로 for 문을 사용하여 작성한다.
def avg_3_apply(col):
    sum = 0
    for item in col:
        print(item)
        sum += item
    return sum / df.shape[0]

In [33]:
df.apply(avg_3_apply)

10
20
30
20
30
40


a    20.0
b    30.0
dtype: float64

In [34]:
# 행방향을 처리하는 함수는 df.shape[1] 로 변경하면 된다.
def avg_2_apply(row):
    sum = 0
    for item in row:
        print(item)
        sum += item
    return sum / df.shape[1]

df.apply(avg_2_apply, axis=1)

10
20
20
30
30
40


0    15.0
1    25.0
2    35.0
dtype: float64

## apply 메서드 사용하기 - 고급

### DataFrame 의 누락값 처리 후 apply 메서드 적용

In [35]:
# 누락값 처리 - 열방향

import seaborn as sns

titanic = sns.load_dataset('titanic')

In [36]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 63.0+ KB


In [40]:
import numpy as np

# 누락값의 개수를 반환하는 count_missing 함수

def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

In [41]:
# apply 메서드에 count_missing 함수를 전달
cmis_col = titanic.apply(count_missing)
cmis_col

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [42]:
# 누락값의 비율을 계산하는 prop_missing 함수
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

In [43]:
pmis_col = titanic.apply(prop_missing)
pmis_col

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64

In [44]:
# props_missing 함수를 이용하여 데이터의 비율을 구한다.
def prop_complete(vec):
    return 1 - prop_missing(vec)

In [45]:
pcom_col = titanic.apply(prop_complete)
pcom_col

survived       1.000000
pclass         1.000000
sex            1.000000
age            0.801347
sibsp          1.000000
parch          1.000000
fare           1.000000
embarked       0.997755
class          1.000000
who            1.000000
adult_male     1.000000
deck           0.227834
embark_town    0.997755
alive          1.000000
alone          1.000000
dtype: float64

### 누락값 처리하기 - 행방향

In [53]:
cmis_row = titanic.apply(count_missing, axis=1)
cmis_row.head()

0    1
1    0
2    1
3    0
4    1
dtype: int64

In [50]:
pmis_row = titanic.apply(prop_missing, axis=1)
pmis_row.head()

0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64

In [52]:
pcom_row = titanic.apply(prop_complete, axis=1)
pcom_row.head()

0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64

In [54]:
# 누락값의 개수를 구하여 titanic 데이터프레임에 추가
titanic['num_missing'] = titanic.apply(count_missing, axis=1)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,num_missing
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,1
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,1


In [55]:
# 누락값이 2개 이상인 데이터 추출
titanic.loc[titanic.num_missing > 1, :].sample(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,num_missing
229,0,3,female,NaN,3,1,25.4667,S,Third,woman,False,NaN,Southampton,no,False,2
584,0,3,male,NaN,0,0,8.7125,C,Third,man,True,NaN,Cherbourg,no,True,2
596,1,2,female,NaN,0,0,33.0000,S,Second,woman,False,NaN,Southampton,yes,True,2
667,0,3,male,NaN,0,0,7.7750,S,Third,man,True,NaN,Southampton,no,True,2
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,2
495,0,3,male,NaN,0,0,14.4583,C,Third,man,True,NaN,Cherbourg,no,True,2
126,0,3,male,NaN,0,0,7.7500,Q,Third,man,True,NaN,Queenstown,no,True,2
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,2
250,0,3,male,NaN,0,0,7.2500,S,Third,man,True,NaN,Southampton,no,True,2
768,0,3,male,NaN,1,0,24.1500,Q,Third,man,True,NaN,Queenstown,no,False,2
